In [1]:
!pip install -q --upgrade openai python-dotenv

In [2]:
!pip install gradio

   ---------------------------------------- 0.0/60.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/60.4 MB ? eta -:--:--
   ---------------------------------------- 0.5/60.4 MB 1.9 MB/s eta 0:00:33
    --------------------------------------- 1.0/60.4 MB 1.9 MB/s eta 0:00:32
    --------------------------------------- 1.3/60.4 MB 1.8 MB/s eta 0:00:33
   - -------------------------------------- 1.8/60.4 MB 1.8 MB/s eta 0:00:32
   - -------------------------------------- 2.1/60.4 MB 1.8 MB/s eta 0:00:32
   - -------------------------------------- 2.6/60.4 MB 1.8 MB/s eta 0:00:32
   - -------------------------------------- 2.9/60.4 MB 1.8 MB/s eta 0:00:32
   -- ------------------------------------- 3.4/60.4 MB 1.8 MB/s eta 0:00:31
   -- ------------------------------------- 3.7/60.4 MB 1.8 MB/s eta 0:00:31
   -- ------------------------------------- 4.2/60.4 MB 1.9 MB/s eta 0:00:31
   --- ------------------------------------ 4.7/60.4 MB 1.9 MB/s eta 0:00:30
   --- ------

  You can safely remove it manually.


In [3]:
import os
from IPython.display import display,Markdown
from openai import OpenAI 
from dotenv import load_dotenv
load_dotenv()
openai_api_key=os.getenv("OPENAI_API_KEY")
openai_client=OpenAI(api_key=openai_api_key,base_url="https://openrouter.ai/api/v1")
print("OpenAI client successfully configured.")
print("First 5 characters of API key:", openai_api_key[:5])



OpenAI client successfully configured.
First 5 characters of API key: sk-or


In [5]:
def print_markdown(text):
    "display text"
    display(Markdown(text))

**BUILD A BASIC AI TUTOR FUNCTION (NO GRADIO YET)**

In [7]:
def get_ai_tutor_response(user_question):
    system_prompt="You are a helpful and patient AI Tutor. Explain concepts clearly and concisely."
    try:
        response=openai_client.chat.completions.create(
            model="mistralai/mistral-small-3.2-24b-instruct:free",
            messages=[{"role":"system","content":system_prompt},{"role":"user","content":user_question},],
              temperature=0.7
        )
        ai_response=response.choices[0].message.content
        return ai_response

    except Exception as e:
                print(f"An error occurred: {e}")

In [10]:
test_question="Could you explain the concept of functions in Python and their purpose in programming?"
print_markdown(f"Asking AI Tutor {test_question}")
tutor_answer=get_ai_tutor_response(test_question)
print_markdown("\n AI Tutor Response \n")
print_markdown(tutor_answer)

Asking AI Tutor Could you explain the concept of functions in Python and their purpose in programming?


 AI Tutor Response 


Absolutely, I'd be happy to explain functions in Python and their purpose in programming!

### What is a Function?
A function is a reusable block of code that performs a specific task. Functions help in organizing code into smaller, manageable, and reusable pieces. They can take inputs (called arguments), perform some operations, and optionally return outputs (called return values).

### Purpose of Functions
1. **Reusability**: Functions allow you to write code once and use it multiple times. This reduces redundancy and makes the code easier to maintain.
2. **Modularity**: Breaking down a large program into smaller functions makes it easier to understand, debug, and test.
3. **Abstraction**: Functions hide the implementation details and expose only what is necessary. This makes the code easier to read and understand.
4. **Organization**: Functions help in organizing code logically, making it more structured and easier to navigate.

### Defining a Function in Python
In Python, you can define a function using the `def` keyword followed by the function name, parentheses `()`, and a colon `:`. The code block inside the function is indented.

```python
def function_name(parameters):
    # Code to execute
    return return_value
```

### Example
Here's a simple example of a function that adds two numbers:

```python
def add_numbers(a, b):
    result = a + b
    return result
```

### Calling a Function
To use a function, you call it by its name followed by parentheses. You can pass arguments inside the parentheses if the function requires them.

```python
sum_result = add_numbers(3, 5)
print(sum_result)  # Output: 8
```

### Key Components
1. **Function Name**: The name of the function (e.g., `add_numbers`).
2. **Parameters**: Variables listed inside the parentheses (e.g., `a` and `b`). They are optional.
3. **Return Value**: The value that the function returns using the `return` statement. This is also optional.
4. **Function Body**: The indented block of code that defines what the function does.

### Default Parameters
You can also define default values for parameters, which are used if no argument is provided.

```python
def greet(name, message="Hello"):
    print(f"{message}, {name}!")

greet("Alice")  # Output: Hello, Alice!
greet("Bob", "Hi")  # Output: Hi, Bob!
```

### Scope of Variables
Variables defined inside a function are local to that function and cannot be accessed outside it unless returned. Variables defined outside functions are global and can be accessed anywhere.

### Built-in Functions
Python comes with many built-in functions like `print()`, `len()`, `sum()`, etc., which you can use directly.

### Conclusion
Functions are a fundamental concept in programming that help in writing clean, efficient, and maintainable code. They encapsulate a specific task, making the code more modular and easier to understand.

**BUILD AN INTERACTIVE INTERFACE USING GRADIO**

In [11]:
import gradio as gr

C:\Users\elwady\MiniConda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
ai_tutor_interface_simple=gr.Interface(
    fn=get_ai_tutor_response,
    inputs=gr.Textbox(lines=2,placeholder="Enter Your Question",label="Your Question"),
    outputs=gr.Textbox(label="AI Tutor Answer",lines=10),
    title="Simple AI Tutor🤖",
    description="Enter your question below and the AI Tutor will provide an explanation",
    allow_flagging="never"
)
ai_tutor_interface_simple.launch()

C:\Users\elwady\MiniConda3\Lib\site-packages\gradio\interface.py:418: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


**ADD Streaming**

In [20]:
def stream_ai_tutor_response(user_question):
    system_prompt="You are a helpful and patient AI Tutor. Explain concepts clearly and concisely."

    try:
        stream=openai_client.chat.completions.create(
            model="mistralai/mistral-small-3.2-24b-instruct:free",
            messages=[{"role":"system","content":system_prompt},{"role":"user","content":user_question}],
            temperature=0.7,
            stream=True
        )

        full_response=""
        for chunk in stream:
            if chunk.choices[0].delta and chunk.choices[0].delta.content:
                text_chunk=chunk.choices[0].delta.content
                full_response+=text_chunk
                yield full_response  #This makes the text appear to be typing in real-time

    except Exception as e:
                print(f"An error occurred during streaming: {e}")

In [22]:
ai_tutor_streaming=gr.Interface(
    fn=stream_ai_tutor_response,
    inputs=gr.Textbox(lines=2,placeholder="Enter Your Question",label="Your Question"),
    outputs=gr.Markdown( label = "AI Tutor's Answer (Streaming)", container = True, height = 250),
    description="Enter your question below and the AI Tutor will provide an explanation",
    title="Simple Streaming AI Tutor🤖",
    allow_flagging="never"

)
ai_tutor_streaming.launch()

C:\Users\elwady\MiniConda3\Lib\site-packages\gradio\interface.py:418: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


**ADD Slider**

In [23]:
explanation_levels={
        1: "like I'm 5 years old",
    2: "like I'm 10 years old",
    3: "like a high school student",
    4: "like a college student",
    5: "like an expert in the field",
}

In [33]:
def ai_tutor_stream_response_with_levels(user_question,explanation_level_value):
    level_description=explanation_levels.get(
        explanation_level_value
    )
    system_prompt=f"You are a helpful and patient AI Tutor {level_description}"
    try:
        stream=openai_client.chat.completions.create(
            model="mistralai/mistral-small-3.2-24b-instruct:free",
            messages=[{"role":"system","content":system_prompt},{"role":"user","content":user_question}],
            temperature=0.7,
            stream=True
        )
        full_response=""
        for chunk in stream:
            if chunk.choices[0].delta and chunk.choices[0].delta.content:
             text_chunk=chunk.choices[0].delta.content
             full_response+=text_chunk

            yield full_response
    except Exception as e:
           print(f"An error occurred during streaming: {e}")
        
        

In [34]:
ai_tutor_slider=gr.Interface(
    fn=ai_tutor_stream_response_with_levels,
    inputs=[    gr.Textbox(lines = 3, placeholder = "Ask the AI Tutor a question...", label = "Your Question"),
           gr.Slider( minimum = 1,
            maximum = 5,
            step = 1,  # Only allow whole numbers
            value = 3,  # Default level (high school)
            label = "Explanation Level"),  # Label for the slider)]
           ],
      outputs = gr.Markdown(label = "AI Tutor's Explanation (Streaming)", container = True, height = 250),
    title = "🎓 Advanced AI Tutor",
    description = "Ask a question and select the desired level of explanation using the slider.",
    allow_flagging = "never",
)
ai_tutor_slider.launch()

C:\Users\elwady\MiniConda3\Lib\site-packages\gradio\interface.py:418: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.
